# Crop Price Forecasting Model Development

This notebook focuses on developing and training models for crop price forecasting. We'll use multiple approaches:
1. SARIMAX for baseline time series forecasting
2. Prophet for trend and seasonality modeling
3. XGBoost for feature-based prediction
4. Ensemble approach combining multiple models

We'll save the trained models as PKL files for production use.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import json

# Time series libraries
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import optuna

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set random seed
np.random.seed(42)

# Create models directory if it doesn't exist
Path('../models/pkl').mkdir(parents=True, exist_ok=True)

In [ ]:
# Load processed data
data = pd.read_csv('../data/processed/market_data_with_features.csv', parse_dates=['date'])
print("Loaded data shape:", data.shape)

# Load analysis results
with open('../data/processed/analysis_results.json', 'r') as f:
    analysis_results = json.load(f)

# Create train-test split
test_size = 60  # Last 60 days for testing
train = data[:-test_size]
test = data[-test_size:]

print("\nTrain-Test Split:")
print(f"Training data: {train.shape}")
print(f"Test data: {test.shape}")

In [ ]:
# SARIMAX Model
def train_sarimax(train_data):
    # Prepare data
    y = train_data.set_index('date')['modal_price']
    
    # Train model
    model = SARIMAX(y, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
    results = model.fit(disp=False)
    
    return results

# Train SARIMAX
print("Training SARIMAX model...")
sarimax_model = train_sarimax(train)

# Save model
with open('../models/pkl/sarimax_model.pkl', 'wb') as f:
    pickle.dump(sarimax_model, f)
print("Saved SARIMAX model")

# Generate SARIMAX predictions
sarimax_pred = sarimax_model.get_forecast(steps=len(test))
sarimax_forecast = pd.DataFrame({
    'date': test['date'],
    'predicted': sarimax_pred.predicted_mean,
    'lower': sarimax_pred.conf_int()['lower modal_price'],
    'upper': sarimax_pred.conf_int()['upper modal_price']
})

# Plot results
fig = go.Figure()

# Actual values
fig.add_trace(go.Scatter(x=data['date'], y=data['modal_price'], name='Actual', mode='lines'))

# Predictions
fig.add_trace(go.Scatter(x=sarimax_forecast['date'], y=sarimax_forecast['predicted'], 
                        name='SARIMAX Forecast', mode='lines', line=dict(dash='dash')))

# Confidence intervals
fig.add_trace(go.Scatter(x=sarimax_forecast['date'], y=sarimax_forecast['upper'],
                        name='Upper CI', line=dict(width=0)))
fig.add_trace(go.Scatter(x=sarimax_forecast['date'], y=sarimax_forecast['lower'],
                        name='Lower CI', fill='tonexty', line=dict(width=0)))

fig.update_layout(title='SARIMAX Model Forecast', xaxis_title='Date', yaxis_title='Price')
fig.show()

# Calculate metrics
sarimax_rmse = np.sqrt(mean_squared_error(test['modal_price'], sarimax_forecast['predicted']))
sarimax_mae = mean_absolute_error(test['modal_price'], sarimax_forecast['predicted'])
print(f"\nSARIMAX Performance Metrics:")
print(f"RMSE: {sarimax_rmse:.2f}")
print(f"MAE: {sarimax_mae:.2f}")

In [ ]:
# Prophet Model
def train_prophet(train_data):
    # Prepare data for Prophet
    prophet_data = train_data[['date', 'modal_price']].rename(columns={
        'date': 'ds',
        'modal_price': 'y'
    })
    
    # Initialize and train model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05
    )
    model.fit(prophet_data)
    
    return model

# Train Prophet
print("Training Prophet model...")
prophet_model = train_prophet(train)

# Save model
with open('../models/pkl/prophet_model.pkl', 'wb') as f:
    pickle.dump(prophet_model, f)
print("Saved Prophet model")

# Generate Prophet predictions
future_dates = prophet_model.make_future_dataframe(periods=len(test), freq='D')
prophet_forecast = prophet_model.predict(future_dates)
prophet_forecast = prophet_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(len(test))

# Plot results
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['date'], y=data['modal_price'], name='Actual', mode='lines'))
fig.add_trace(go.Scatter(x=prophet_forecast['ds'], y=prophet_forecast['yhat'], 
                        name='Prophet Forecast', mode='lines', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=prophet_forecast['ds'], y=prophet_forecast['yhat_upper'],
                        name='Upper CI', line=dict(width=0)))
fig.add_trace(go.Scatter(x=prophet_forecast['ds'], y=prophet_forecast['yhat_lower'],
                        name='Lower CI', fill='tonexty', line=dict(width=0)))

fig.update_layout(title='Prophet Model Forecast', xaxis_title='Date', yaxis_title='Price')
fig.show()

# Calculate metrics
prophet_rmse = np.sqrt(mean_squared_error(test['modal_price'], prophet_forecast['yhat']))
prophet_mae = mean_absolute_error(test['modal_price'], prophet_forecast['yhat'])
print(f"\nProphet Performance Metrics:")
print(f"RMSE: {prophet_rmse:.2f}")
print(f"MAE: {prophet_mae:.2f}")

In [ ]:
# XGBoost Model with Feature Engineering
def prepare_features(df):
    # Use lag features and market indicators
    feature_cols = [col for col in df.columns if col.startswith(('MA_', 'price_volatility_', 'momentum_'))
                   or col in ['temp_max', 'temp_min', 'precipitation', 'humidity']]
    
    X = df[feature_cols]
    y = df['modal_price']
    
    return X, y

def train_xgboost(X_train, y_train):
    # Define parameter optimization
    def objective(trial):
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
            'n_estimators': trial.suggest_int('n_estimators', 100, 500),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'objective': 'reg:squarederror'
        }
        
        model = xgb.XGBRegressor(**params)
        model.fit(X_train, y_train)
        
        # Use last 30 days of training data for validation
        val_preds = model.predict(X_train[-30:])
        val_score = mean_squared_error(y_train[-30:], val_preds, squared=False)
        
        return val_score
    
    # Run optimization
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)
    
    # Train final model with best parameters
    best_params = study.best_params
    final_model = xgb.XGBRegressor(**best_params)
    final_model.fit(X_train, y_train)
    
    return final_model

# Prepare features
X_train, y_train = prepare_features(train)
X_test, y_test = prepare_features(test)

# Train XGBoost
print("Training XGBoost model...")
xgb_model = train_xgboost(X_train, y_train)

# Save model
with open('../models/pkl/xgboost_model.pkl', 'wb') as f:
    pickle.dump(xgb_model, f)
print("Saved XGBoost model")

# Generate predictions
xgb_pred = xgb_model.predict(X_test)

# Plot results
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['date'], y=data['modal_price'], name='Actual', mode='lines'))
fig.add_trace(go.Scatter(x=test['date'], y=xgb_pred, name='XGBoost Forecast', mode='lines', line=dict(dash='dash')))

fig.update_layout(title='XGBoost Model Forecast', xaxis_title='Date', yaxis_title='Price')
fig.show()

# Calculate metrics
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_mae = mean_absolute_error(y_test, xgb_pred)
print(f"\nXGBoost Performance Metrics:")
print(f"RMSE: {xgb_rmse:.2f}")
print(f"MAE: {xgb_mae:.2f}")

# Feature importance
importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Important Features:")
print(importance_df.head(10))

In [ ]:
# Ensemble Model - Weighted Average of All Models
def create_ensemble_prediction(sarimax_pred, prophet_pred, xgb_pred, weights=None):
    if weights is None:
        # Equal weights by default
        weights = [1/3, 1/3, 1/3]
    
    ensemble_pred = (weights[0] * sarimax_pred + 
                    weights[1] * prophet_pred + 
                    weights[2] * xgb_pred)
    return ensemble_pred

# Get all predictions for the test period
ensemble_weights = [0.4, 0.3, 0.3]  # Can be optimized based on validation performance
ensemble_pred = create_ensemble_prediction(
    sarimax_forecast['predicted'].values,
    prophet_forecast['yhat'].values,
    xgb_pred,
    weights=ensemble_weights
)

# Create ensemble forecast dataframe
ensemble_forecast = pd.DataFrame({
    'date': test['date'],
    'actual': test['modal_price'],
    'ensemble_pred': ensemble_pred,
    'sarimax_pred': sarimax_forecast['predicted'].values,
    'prophet_pred': prophet_forecast['yhat'].values,
    'xgb_pred': xgb_pred
})

# Save ensemble metadata
ensemble_metadata = {
    'weights': ensemble_weights,
    'models': ['sarimax', 'prophet', 'xgboost'],
    'performance': {
        'sarimax_rmse': float(sarimax_rmse),
        'prophet_rmse': float(prophet_rmse),
        'xgb_rmse': float(xgb_rmse),
        'ensemble_rmse': float(np.sqrt(mean_squared_error(test['modal_price'], ensemble_pred)))
    }
}

with open('../models/pkl/ensemble_metadata.json', 'w') as f:
    json.dump(ensemble_metadata, f, indent=4)

# Plot all models together
fig = go.Figure()

fig.add_trace(go.Scatter(x=test['date'], y=test['modal_price'], name='Actual', mode='lines'))
fig.add_trace(go.Scatter(x=test['date'], y=ensemble_pred, name='Ensemble', mode='lines', 
                        line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=test['date'], y=sarimax_forecast['predicted'], 
                        name='SARIMAX', mode='lines', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=test['date'], y=prophet_forecast['yhat'], 
                        name='Prophet', mode='lines', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=test['date'], y=xgb_pred, 
                        name='XGBoost', mode='lines', line=dict(dash='dash')))

fig.update_layout(
    title='Model Comparison on Test Set',
    xaxis_title='Date',
    yaxis_title='Price',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)
fig.show()

# Print final metrics
print("\nFinal Model Performance Metrics (RMSE):")
print("-" * 40)
print(f"SARIMAX:  {sarimax_rmse:.2f}")
print(f"Prophet:  {prophet_rmse:.2f}")
print(f"XGBoost:  {xgb_rmse:.2f}")
print(f"Ensemble: {np.sqrt(mean_squared_error(test['modal_price'], ensemble_pred)):.2f}")